In [16]:
library(keras)
library(tidyverse)
library(glue)


Attaching package: ‘glue’


The following object is masked from ‘package:dplyr’:

    collapse




In [4]:
url="https://cssbook.net/d/dutch_sentiment.csv"
d = read_csv(url)
head(d)


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  id = col_double(),
  value = col_double(),
  lemmata = col_character()
)




id,value,lemmata
<dbl>,<dbl>,<chr>
10007,0,Rabobank voorspellen flink stijging hypotheekrente
10027,0,D66 willen reserve provincie aanspreken voor groei
10037,1,UWV dit jaar veel baan
10059,1,proost op geslaagd beursgang bols
10099,0,helft werknemer gaan na 65ste met pensioen
10101,1,Europa groeien voorzichtig dankzij laag energieprijs


In [6]:
text_vectorization = layer_text_vectorization(
    max_tokens=10000, output_sequence_length=50)
adapt(text_vectorization, d$lemmata)

input = layer_input(shape=1, dtype = "string")
output = input %>% 
  text_vectorization() %>% 
  layer_embedding(input_dim = 10000 + 1, 
                  output_dim = 16) %>%
  layer_conv_1d(filters=128, kernel_size=3, 
                activation="relu") %>%
  layer_global_max_pooling_1d() %>%
  layer_dense(units = 64, activation = "relu") %>%
  layer_dense(units = 1, activation = "tanh")

model = keras_model(input, output)
model

Model
Model: "functional_3"
________________________________________________________________________________
Layer (type)                        Output Shape                    Param #     
input_2 (InputLayer)                [(None, 1)]                     0           
________________________________________________________________________________
text_vectorization_1 (TextVectoriza (None, 50)                      0           
________________________________________________________________________________
embedding_1 (Embedding)             (None, 50, 16)                  160016      
________________________________________________________________________________
conv1d_1 (Conv1D)                   (None, 48, 128)                 6272        
________________________________________________________________________________
global_max_pooling1d_1 (GlobalMaxPo (None, 128)                     0           
________________________________________________________________________________


In [24]:
# Split data into train and test
d_train = d %>% slice_sample(n=4000)
d_test = d %>% anti_join(d_train)



# Train model
compile(model, loss = "binary_crossentropy",
        optimizer = "adam", metrics = "accuracy")
fit(model, d_train$lemmata, d_train$value,
  epochs = 10, batch_size = 512, 
  validation_split = 0.2)
# Validate against test data
eval=evaluate(model, d_test$lemmata, d_test$value)
print(glue("Accuracy: {eval['accuracy']}"))

Joining, by = c("id", "value", "lemmata")



Accuracy: 0.431955218315125
